In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# Simulation of the BB84 key distribution protocol.

# Use a quantum circuit to generate a random bit.
def random():
    q = QuantumCircuit(1) 
    q.h(0) 
    q.measure_all() 
    backend = BasicSimulator()
    compiled = transpile(q, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)
    return counts.get("1",0)

# Generate a list of n random bits
def randomList(n):
    bs = []
    for i in range(n):
        bs = bs + [random()] 
    return bs 
    
# The length of the list of bits that Alice will generate
N = 20

# Alice generates a random list of N bits
aliceBits = randomList(N) 

# Alice generates a random list to determine her choice of basis for each bit
# 0 means the standard basis, 1 means the diagonal basis
aliceBases = randomList(N) 
print("Alice bases: %s" % aliceBases) 

# Bob generates a random list to determine his choice of measurement basis for each qubit
bobBases = randomList(N) 
print("Bob bases:   %s" % bobBases) 

print("Alice bits:  %s" % aliceBits) 

# Bob will measure each qubit, getting a list of bits. Initialise it now and extend it later
bobBits = [] 

# Alice represents each of her bits by a qubit.
# Bob will measure each qubit.
# We can do this by constructing a circuit for Alice's qubit, then Bob can add the measurement and simulate the circuit.
# Do it in a loop, one qubit at a time

for i in range(N):
    # The first part of the loop implements Alice's actions
    q = QuantumCircuit(1) 
    # The qubit starts in state |0>
    if aliceBits[i] == 1:
        # Flip the qubit
        q.x(0) 
    # The qubit now represents Alice's ith bit in the standard basis
    if aliceBases[i] == 1: 
        # Change to the diagonal basis
        q.h(0) 

    # The next part of the loop implements Bob's actions
    if bobBases[i] == 1:
        # Bob measures this qubit in the diagonal basis. To achieve this, apply H before measuring
        q.h(0)
    # Now Bob measures the qubit (in the standard basis)
    q.measure_all() 
    backend = BasicSimulator()
    compiled = transpile(q, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)
    bit = counts.get("1",0)
    bobBits = bobBits + [bit] 

print("Bob bits:    %s" % bobBits) 

# Now construct the shared key by comparing the basis choices
sharedKey = [] 

for i in range(N):
    if aliceBases[i] == bobBases[i]: 
        # Alice's and Bob's bits should be the same - check
        if aliceBits[i] != bobBits[i]:
            print("Error: bits don't match at position %i" % i) 
        else:
            sharedKey = sharedKey + [aliceBits[i]] 

print("Shared key (length %i): %s" % (len(sharedKey), sharedKey)) 


Alice bases: [0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0]
Bob bases:   [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
Alice bits:  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1]
Bob bits:    [0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0]
Shared key (length 24): [0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]
